In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

# matplotlib.rcParams['font.family'] = 'Malgun Gothic'  # windows
matplotlib.rcParams['font.family'] = 'AppleGothic'  # Mac
matplotlib.rcParams['font.size'] = 15  # 글자크기
matplotlib.rcParams['axes.unicode_minus'] = False # 한글 폰트 사용시 마이너스 꺠지지않게 해줌

In [11]:
train_df = pd.read_csv('./DF_Preprocessing/train_df.csv')
test_df = pd.read_csv('./DF_Preprocessing/test_df.csv')

In [12]:
train_df

,자치구,계약일,건물면적(㎡),층,KOSPI종가,Fed기준금리,국제유가(Dubai)($/배럴),부동산거래량,국고채3년(평균),국고채5년(평균),국고채10년(평균),회사채3년(평균),CD91물(평균),"콜금리(1일물,평균)",BOK기준금리,건물연식,GDP대비가계부채비율,물건금액(만원)
0,노원구,2016-01-01,39.96,10,1961.310059,0.34,26.9,9894,1.63,1.77,2.03,2.10,1.67,1.49,1.5,25,83.300000,18000
1,성동구,2016-01-01,75.37,10,1961.310059,0.34,26.9,9894,1.63,1.77,2.03,2.10,1.67,1.49,1.5,26,83.300000,55200
2,금천구,2016-01-01,84.83,3,1961.310059,0.34,26.9,9894,1.63,1.77,2.03,2.10,1.67,1.49,1.5,14,83.300000,36800
3,은평구,2016-01-01,101.97,9,1961.310059,0.34,26.9,9894,1.63,1.77,2.03,2.10,1.67,1.49,1.5,6,83.300000,67600
4,노원구,2016-01-01,84.59,10,1961.310059,0.34,26.9,9894,1.63,1.77,2.03,2.10,1.67,1.49,1.5,26,83.300000,30300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453165,성동구,2020-12-31,84.30,3,2873.469971,0.09,49.8,30495,0.97,1.32,1.68,2.21,0.66,0.51,0.5,4,103.933333,171000
453166,성북구,2020-12-31,59.98,2,2873.469971,0.09,49.8,30495,0.97,1.32,1.68,2.21,0.66,0.51,0.5,19,103.933333,47000
453167,용산구,2020-12-31,84.21,7,2873.469971,0.09,49.8,30495,0.97,1.32,1.68,2.21,0.66,0.51,0.5,9,103.933333,175000
453168,은평구,2020-12-31,84.69,1,2873.469971,0.09,49.8,30495,0.97,1.32,1.68,2.21,0.66,0.51,0.5,16,103.933333,59700


In [13]:
# X, y 나누기
X = train_df.drop('물건금액(만원)', axis = 1)
y = train_df['물건금액(만원)']

### 피쳐 스케일링

In [15]:
def myscaler(df):
    from sklearn.preprocessing import MinMaxScaler

    df_num = df.select_dtypes([np.number])
    num_col = list(df_num.columns)

    # Scaler 객체 생성
    scaler = MinMaxScaler()
    x_num_scaled = scaler.fit_transform(df_num)

    # print('\t\t(min, max) (mean, std)')
    print('Scaled (%.2f, %.2f) (%.2f, %.2f)' %(x_num_scaled.min(), x_num_scaled.max(), x_num_scaled.mean(), x_num_scaled.std()))
    x_num_scaled = pd.DataFrame(x_num_scaled, columns=num_col)

    X_scaled = df.copy()
    for i in x_num_scaled:
        X_scaled[i] = x_num_scaled[i]

    return X_scaled

In [17]:
X = myscaler(X)

Scaled (0.00, 1.00) (0.44, 0.27)


In [20]:
X[['GDP대비가계부채비율']].describe()

,GDP대비가계부채비율
count,453170.000000
mean,0.387582
std,0.250948
min,0.000000
25%,0.203554
50%,0.310178
75%,0.591276
max,1.000000


### 타겟 변수 로그 변환

In [24]:
import numpy as np

# 로그 변환 적용
y = np.log(y)
y

0         2.282191
1         2.390479
2         2.352637
3         2.408868
4         2.333977
            ...   
453165    2.489016
453166    2.375641
453167    2.490934
453168    2.397630
453169    2.411615
Name: 물건금액(만원), Length: 453170, dtype: float64

In [26]:
X_train = X.select_dtypes([np.number])

In [28]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453170 entries, 0 to 453169
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   건물면적(㎡)            453170 non-null  float64
 1   층                  453170 non-null  float64
 2   KOSPI종가            453170 non-null  float64
 3   Fed기준금리            453170 non-null  float64
 4   국제유가(Dubai)($/배럴)  453170 non-null  float64
 5   부동산거래량             453170 non-null  float64
 6   국고채3년(평균)          453170 non-null  float64
 7   국고채5년(평균)          453170 non-null  float64
 8   국고채10년(평균)         453170 non-null  float64
 9   회사채3년(평균)          453170 non-null  float64
 10  CD91물(평균)          453170 non-null  float64
 11  콜금리(1일물,평균)        453170 non-null  float64
 12  BOK기준금리            453170 non-null  float64
 13  건물연식               453170 non-null  float64
 14  GDP대비가계부채비율        453170 non-null  float64
dtypes: float64(15)
memory usage: 51.9 MB


In [29]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                 3.005e+04
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        16:56:36   Log-Likelihood:             8.2791e+05
No. Observations:              453170   AIC:                        -1.656e+06
Df Residuals:                  453154   BIC:                        -1.656e+06
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.2786      0.001   2025.207      0.000       2.276       2.281
건물면적(㎡)               0.4758      0.001    592.670      0.000       0.474       0.477
층                     0.0602      0.001     86.388      0.000       0.059       0.062
KOSPI종가               0.0276      0.001     27.741      0.000       0.026       0.030
Fed기준금리               0.0070      0.001      9.338      0.000       0.006       0.008
국제유가(Dubai)($/배럴)     0.0113      0.001     16.039      0.000       0.010       0.013
부동산거래량               -0.0067      0.000    -17.114      0.000      -0.008      -0.006
국고채3년(평균)            -0.0720      0.006    -12.825      0.000      -0.083      -0.061
국고채5년(평균)             0.0701      0.008      9.021      0.000       0.055       0.085
국고채10년(평균)           -0.0391      0.003    -12.608      0.000      -0.045      -0.033
회사채3년(평균)             0.0049      0.001      4.423      0.000       0.003       0.007
CD91물(평균)             0.0087      0.002      4.407      0.000       0.005       0.013
콜금리(1일물,평균)          -0.0013      0.002     -0.567      0.571      -0.006       0.003
BOK기준금리               0.0039      0.002      2.376      0.017       0.001       0.007
건물연식                 -0.0024      0.000     -6.450      0.000      -0.003      -0.002
GDP대비가계부채비율           0.0438      0.001     41.708      0.000       0.042       0.046
==============================================================================
Omnibus:                     7750.377   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9046.573
Skew:                           0.271   Prob(JB):                         0.00
Kurtosis:                       3.431   Cond. No.                         359.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 콜금리 컬럼 삭제

In [44]:
X_train_1 = X_train.drop('콜금리(1일물,평균)', axis = 1)

In [45]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train_1, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                 3.219e+04
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:26:59   Log-Likelihood:             8.2791e+05
No. Observations:              453170   AIC:                        -1.656e+06
Df Residuals:                  453155   BIC:                        -1.656e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.2785      0.001   2090.131      0.000       2.276       2.281
건물면적(㎡)               0.4758      0.001    592.673      0.000       0.474       0.477
층                     0.0602      0.001     86.390      0.000       0.059       0.062
KOSPI종가               0.0275      0.001     28.678      0.000       0.026       0.029
Fed기준금리               0.0068      0.001      9.740      0.000       0.005       0.008
국제유가(Dubai)($/배럴)     0.0113      0.001     16.094      0.000       0.010       0.013
부동산거래량               -0.0067      0.000    -17.773      0.000      -0.007      -0.006
국고채3년(평균)            -0.0711      0.005    -13.184      0.000      -0.082      -0.061
국고채5년(평균)             0.0688      0.007      9.232      0.000       0.054       0.083
국고채10년(평균)           -0.0386      0.003    -12.974      0.000      -0.044      -0.033
회사채3년(평균)             0.0051      0.001      4.751      0.000       0.003       0.007
CD91물(평균)             0.0080      0.001      5.449      0.000       0.005       0.011
BOK기준금리               0.0034      0.001      2.523      0.012       0.001       0.006
건물연식                 -0.0024      0.000     -6.448      0.000      -0.003      -0.002
GDP대비가계부채비율           0.0440      0.001     43.186      0.000       0.042       0.046
==============================================================================
Omnibus:                     7750.840   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9047.524
Skew:                           0.271   Prob(JB):                         0.00
Kurtosis:                       3.431   Cond. No.                         326.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_engineering_XbyVIF(X_train_1):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train_1.values,i)
                         for i in range(X_train_1.shape[1])]
    vif["Feature"] = X_train_1.columns
    return vif
vif = feature_engineering_XbyVIF(X_train_1)
print(vif)

     VIF_Factor            Feature
0      6.208594            건물면적(㎡)
1      5.049965                  층
2     73.996410            KOSPI종가
3     25.117975            Fed기준금리
4     51.682344  국제유가(Dubai)($/배럴)
5      8.161876             부동산거래량
6   1914.239436          국고채3년(평균)
7   2615.171930          국고채5년(평균)
8    483.637043         국고채10년(평균)
9     80.299953          회사채3년(평균)
10   249.741920          CD91물(평균)
11   233.748568            BOK기준금리
12     5.542219               건물연식
13    44.189900        GDP대비가계부채비율


### 국고채 5년 컬럼 삭제

In [47]:
X_train_2 = X_train_1.drop('국고채5년(평균)', axis = 1)

In [48]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train_2, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                 3.466e+04
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:28:17   Log-Likelihood:             8.2787e+05
No. Observations:              453170   AIC:                        -1.656e+06
Df Residuals:                  453156   BIC:                        -1.656e+06
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.2719      0.001   2744.594      0.000       2.270       2.274
건물면적(㎡)               0.4758      0.001    592.669      0.000       0.474       0.477
층                     0.0602      0.001     86.474      0.000       0.059       0.062
KOSPI종가               0.0271      0.001     28.303      0.000       0.025       0.029
Fed기준금리               0.0058      0.001      8.349      0.000       0.004       0.007
국제유가(Dubai)($/배럴)     0.0112      0.001     15.974      0.000       0.010       0.013
부동산거래량               -0.0068      0.000    -18.164      0.000      -0.008      -0.006
국고채3년(평균)            -0.0248      0.002    -12.475      0.000      -0.029      -0.021
국고채10년(평균)           -0.0149      0.002     -9.879      0.000      -0.018      -0.012
회사채3년(평균)             0.0118      0.001     15.077      0.000       0.010       0.013
CD91물(평균)             0.0071      0.001      4.821      0.000       0.004       0.010
BOK기준금리               0.0027      0.001      2.036      0.042       0.000       0.005
건물연식                 -0.0024      0.000     -6.429      0.000      -0.003      -0.002
GDP대비가계부채비율           0.0472      0.001     49.169      0.000       0.045       0.049
==============================================================================
Omnibus:                     7732.534   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9035.427
Skew:                           0.270   Prob(JB):                         0.00
Kurtosis:                       3.432   Cond. No.                         81.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_engineering_XbyVIF(X_train_2):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train_2.values,i)
                         for i in range(X_train_2.shape[1])]
    vif["Feature"] = X_train_2.columns
    return vif
vif = feature_engineering_XbyVIF(X_train_2)
print(vif)

    VIF_Factor            Feature
0     6.137939            건물면적(㎡)
1     5.006628                  층
2    66.232189            KOSPI종가
3    21.392021            Fed기준금리
4    51.458514  국제유가(Dubai)($/배럴)
5     7.866602             부동산거래량
6   382.703159          국고채3년(평균)
7   194.302984         국고채10년(평균)
8    47.501366          회사채3년(평균)
9   211.127741          CD91물(평균)
10  228.130949            BOK기준금리
11    5.476974               건물연식
12   44.025860        GDP대비가계부채비율


### 국고채3년(평균) 컬럼 삭제

In [54]:
X_train_3 = X_train_2.drop('국고채3년(평균)', axis = 1)

In [55]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train_3, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.498
Model:                            OLS   Adj. R-squared:                  0.498
Method:                 Least Squares   F-statistic:                 3.752e+04
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:31:52   Log-Likelihood:             8.2779e+05
No. Observations:              453170   AIC:                        -1.656e+06
Df Residuals:                  453157   BIC:                        -1.655e+06
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.2731      0.001   2763.578      0.000       2.271       2.275
건물면적(㎡)               0.4757      0.001    592.461      0.000       0.474       0.477
층                     0.0602      0.001     86.435      0.000       0.059       0.062
KOSPI종가               0.0255      0.001     26.907      0.000       0.024       0.027
Fed기준금리               0.0046      0.001      6.709      0.000       0.003       0.006
국제유가(Dubai)($/배럴)     0.0081      0.001     12.321      0.000       0.007       0.009
부동산거래량               -0.0071      0.000    -18.801      0.000      -0.008      -0.006
국고채10년(평균)           -0.0292      0.001    -29.569      0.000      -0.031      -0.027
회사채3년(평균)             0.0093      0.001     12.329      0.000       0.008       0.011
CD91물(평균)             0.0037      0.001      2.544      0.011       0.001       0.006
BOK기준금리              -0.0011      0.001     -0.831      0.406      -0.004       0.001
건물연식                 -0.0023      0.000     -6.057      0.000      -0.003      -0.002
GDP대비가계부채비율           0.0509      0.001     55.954      0.000       0.049       0.053
==============================================================================
Omnibus:                     7739.191   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9025.847
Skew:                           0.271   Prob(JB):                         0.00
Kurtosis:                       3.429   Cond. No.                         66.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### BOK기준금리 컬럼 삭제

In [57]:
X_train_4 = X_train_3.drop('BOK기준금리', axis = 1)

In [58]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train_4, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.498
Model:                            OLS   Adj. R-squared:                  0.498
Method:                 Least Squares   F-statistic:                 4.093e+04
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:32:12   Log-Likelihood:             8.2779e+05
No. Observations:              453170   AIC:                        -1.656e+06
Df Residuals:                  453158   BIC:                        -1.655e+06
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.2730      0.001   2797.047      0.000       2.271       2.275
건물면적(㎡)               0.4757      0.001    592.473      0.000       0.474       0.477
층                     0.0602      0.001     86.440      0.000       0.059       0.062
KOSPI종가               0.0252      0.001     30.457      0.000       0.024       0.027
Fed기준금리               0.0044      0.001      6.827      0.000       0.003       0.006
국제유가(Dubai)($/배럴)     0.0080      0.001     12.303      0.000       0.007       0.009
부동산거래량               -0.0070      0.000    -19.398      0.000      -0.008      -0.006
국고채10년(평균)           -0.0289      0.001    -32.228      0.000      -0.031      -0.027
회사채3년(평균)             0.0092      0.001     12.411      0.000       0.008       0.011
CD91물(평균)             0.0028      0.001      2.862      0.004       0.001       0.005
건물연식                 -0.0023      0.000     -6.054      0.000      -0.003      -0.002
GDP대비가계부채비율           0.0514      0.001     69.734      0.000       0.050       0.053
==============================================================================
Omnibus:                     7739.543   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9027.851
Skew:                           0.271   Prob(JB):                         0.00
Kurtosis:                       3.430   Cond. No.                         45.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_engineering_XbyVIF(X_train_4):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train_4.values,i)
                         for i in range(X_train_4.shape[1])]
    vif["Feature"] = X_train_4.columns
    return vif
vif = feature_engineering_XbyVIF(X_train_4)
print(vif)

    VIF_Factor            Feature
0     6.134809            건물면적(㎡)
1     5.003559                  층
2    40.371632            KOSPI종가
3    17.581270            Fed기준금리
4    44.492098  국제유가(Dubai)($/배럴)
5     7.310916             부동산거래량
6    68.979323         국고채10년(평균)
7    42.545317          회사채3년(평균)
8    24.622287          CD91물(평균)
9     5.460097               건물연식
10   22.293228        GDP대비가계부채비율


### 국고채10년(평균) 컬럼 삭제

In [60]:
X_train_5 = X_train_4.drop('국고채10년(평균)', axis = 1)

In [61]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train_5, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.497
Model:                            OLS   Adj. R-squared:                  0.497
Method:                 Least Squares   F-statistic:                 4.482e+04
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:33:44   Log-Likelihood:             8.2727e+05
No. Observations:              453170   AIC:                        -1.655e+06
Df Residuals:                  453159   BIC:                        -1.654e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.2738      0.001   2796.047      0.000       2.272       2.275
건물면적(㎡)               0.4760      0.001    592.232      0.000       0.474       0.478
층                     0.0602      0.001     86.384      0.000       0.059       0.062
KOSPI종가               0.0136      0.001     18.214      0.000       0.012       0.015
Fed기준금리               0.0032      0.001      4.973      0.000       0.002       0.004
국제유가(Dubai)($/배럴)     0.0066      0.001     10.104      0.000       0.005       0.008
부동산거래량               -0.0074      0.000    -20.717      0.000      -0.008      -0.007
회사채3년(평균)            -0.0116      0.000    -31.891      0.000      -0.012      -0.011
CD91물(평균)            -0.0015      0.001     -1.552      0.121      -0.003       0.000
건물연식                 -0.0023      0.000     -6.021      0.000      -0.003      -0.002
GDP대비가계부채비율           0.0653      0.001    109.358      0.000       0.064       0.066
==============================================================================
Omnibus:                     7817.736   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9095.442
Skew:                           0.274   Prob(JB):                         0.00
Kurtosis:                       3.427   Cond. No.                         43.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [62]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_engineering_XbyVIF(X_train_5):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train_5.values,i)
                         for i in range(X_train_5.shape[1])]
    vif["Feature"] = X_train_5.columns
    return vif
vif = feature_engineering_XbyVIF(X_train_5)
print(vif)

   VIF_Factor            Feature
0    6.132936            건물면적(㎡)
1    5.003379                  층
2   29.808919            KOSPI종가
3   17.375049            Fed기준금리
4   44.293439  국제유가(Dubai)($/배럴)
5    7.301537             부동산거래량
6    8.657840          회사채3년(평균)
7   23.141987          CD91물(평균)
8    5.459896               건물연식
9    9.642159        GDP대비가계부채비율


### 국제유가(Dubai)($/배럴) 컬럼 삭제

In [64]:
X_train_6 = X_train_5.drop('국제유가(Dubai)($/배럴)', axis = 1)

In [66]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train_6, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.497
Model:                            OLS   Adj. R-squared:                  0.497
Method:                 Least Squares   F-statistic:                 4.978e+04
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:35:44   Log-Likelihood:             8.2722e+05
No. Observations:              453170   AIC:                        -1.654e+06
Df Residuals:                  453160   BIC:                        -1.654e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           2.2743      0.001   2802.161      0.000       2.273       2.276
건물면적(㎡)         0.4762      0.001    592.395      0.000       0.475       0.478
층               0.0602      0.001     86.326      0.000       0.059       0.062
KOSPI종가         0.0150      0.001     20.616      0.000       0.014       0.016
Fed기준금리         0.0066      0.001     11.983      0.000       0.006       0.008
부동산거래량         -0.0066      0.000    -18.876      0.000      -0.007      -0.006
회사채3년(평균)      -0.0107      0.000    -30.361      0.000      -0.011      -0.010
CD91물(평균)      -0.0010      0.001     -1.078      0.281      -0.003       0.001
건물연식           -0.0023      0.000     -6.173      0.000      -0.003      -0.002
GDP대비가계부채비율     0.0652      0.001    109.148      0.000       0.064       0.066
==============================================================================
Omnibus:                     7831.835   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9109.586
Skew:                           0.274   Prob(JB):                         0.00
Kurtosis:                       3.427   Cond. No.                         41.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [67]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_engineering_XbyVIF(X_train_6):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train_6.values,i)
                         for i in range(X_train_6.shape[1])]
    vif["Feature"] = X_train_6.columns
    return vif
vif = feature_engineering_XbyVIF(X_train_6)
print(vif)

   VIF_Factor      Feature
0    6.128488      건물면적(㎡)
1    5.003247            층
2   27.041731      KOSPI종가
3    9.733684      Fed기준금리
4    6.836086       부동산거래량
5    8.194934    회사채3년(평균)
6   21.823018    CD91물(평균)
7    5.459814         건물연식
8    9.626342  GDP대비가계부채비율


### KOSPI종가 컬럼 삭제

In [74]:
X_train_7 = X_train_6.drop('KOSPI종가', axis = 1)

In [75]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train_7, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.497
Model:                            OLS   Adj. R-squared:                  0.497
Method:                 Least Squares   F-statistic:                 5.589e+04
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:53:56   Log-Likelihood:             8.2701e+05
No. Observations:              453170   AIC:                        -1.654e+06
Df Residuals:                  453161   BIC:                        -1.654e+06
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           2.2852      0.001   3704.383      0.000       2.284       2.286
건물면적(㎡)         0.4762      0.001    592.217      0.000       0.475       0.478
층               0.0602      0.001     86.334      0.000       0.059       0.062
Fed기준금리         0.0118      0.000     24.039      0.000       0.011       0.013
부동산거래량         -0.0061      0.000    -17.578      0.000      -0.007      -0.005
회사채3년(평균)      -0.0051      0.000    -22.795      0.000      -0.005      -0.005
CD91물(평균)      -0.0109      0.001    -13.068      0.000      -0.013      -0.009
건물연식           -0.0024      0.000     -6.355      0.000      -0.003      -0.002
GDP대비가계부채비율     0.0607      0.001    109.062      0.000       0.060       0.062
==============================================================================
Omnibus:                     7748.592   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8998.577
Skew:                           0.273   Prob(JB):                         0.00
Kurtosis:                       3.423   Cond. No.                         31.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [76]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_engineering_XbyVIF(X_train_7):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train_7.values,i)
                         for i in range(X_train_7.shape[1])]
    vif["Feature"] = X_train_7.columns
    return vif
vif = feature_engineering_XbyVIF(X_train_7)
print(vif)

   VIF_Factor      Feature
0    5.982137      건물면적(㎡)
1    4.909607            층
2    9.635258      Fed기준금리
3    6.285199       부동산거래량
4    3.828945    회사채3년(평균)
5   20.129956    CD91물(평균)
6    5.354830         건물연식
7    8.964431  GDP대비가계부채비율


### CD91물(평균) 컬럼 삭제

In [77]:
X_train_8 = X_train_7.drop('CD91물(평균)', axis = 1)

In [78]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train_8, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.496
Method:                 Least Squares   F-statistic:                 6.383e+04
Date:                Tue, 20 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:55:13   Log-Likelihood:             8.2693e+05
No. Observations:              453170   AIC:                        -1.654e+06
Df Residuals:                  453162   BIC:                        -1.654e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           2.2781      0.000   7811.253      0.000       2.278       2.279
건물면적(㎡)         0.4765      0.001    592.636      0.000       0.475       0.478
층               0.0603      0.001     86.381      0.000       0.059       0.062
Fed기준금리         0.0061      0.000     27.211      0.000       0.006       0.007
부동산거래량         -0.0054      0.000    -15.595      0.000      -0.006      -0.005
회사채3년(평균)      -0.0049      0.000    -22.310      0.000      -0.005      -0.005
건물연식           -0.0025      0.000     -6.569      0.000      -0.003      -0.002
GDP대비가계부채비율     0.0669      0.000    230.688      0.000       0.066       0.067
==============================================================================
Omnibus:                     7717.729   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8969.663
Skew:                           0.272   Prob(JB):                         0.00
Kurtosis:                       3.424   Cond. No.                         19.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [79]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_engineering_XbyVIF(X_train_8):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train_8.values,i)
                         for i in range(X_train_8.shape[1])]
    vif["Feature"] = X_train_8.columns
    return vif
vif = feature_engineering_XbyVIF(X_train_8)
print(vif)

   VIF_Factor      Feature
0    5.106132      건물면적(㎡)
1    4.198475            층
2    3.540752      Fed기준금리
3    5.649676       부동산거래량
4    3.615585    회사채3년(평균)
5    4.236322         건물연식
6    5.320428  GDP대비가계부채비율


In [72]:
X_num = X.select_dtypes([np.number])

In [81]:
import numpy as np 
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X_train_8, y, test_size = 0.2)

from sklearn.linear_model import LinearRegression

mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test,y_pred)

print('MSE : {0:}, RMSE : {1: .3f}, MAE:{1: .3f}'.format(mse, rmse, mae))
print('Variance score : {0: .3f}'.format(r2_score(y_test, y_pred)))

MSE : 0.0015256848466321863, RMSE :  0.039, MAE: 0.039
Variance score :  0.497
